In [1]:
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense, InputLayer, Activation
from keras.optimizers import Adam

import tensorflow as tf
import numpy as np

In [2]:
df = pd.read_csv("train.csv",header=None)

In [3]:
df2 = pd.read_json('train.json')

In [4]:
df3 = pd.read_json("test.json")

### Word Embeddings

In [5]:
import nltk
nltk.download('brown')

import gensim
import gensim.downloader
from nltk.corpus import brown
import string

[nltk_data] Downloading package brown to
[nltk_data]     /Users/manojbhadu/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [6]:
sents = brown.sents()
cleaned_sents = []
for i in sents:
    sentence = []
    for j in i :
        if j not in string.punctuation :
            sentence.append(j.lower())
    cleaned_sents.append(sentence)

In [7]:
model_word_emb = gensim.models.FastText(sentences=cleaned_sents,size=100)

# Part1

In [9]:
class feed_forwrd():
    def __init__(self, df_train):
        self.nc1 = df_train[0]
        self.nc2 = df_train[1]
        self.output_class = df_train[2]
        
    def get_embedding(self):
        self.X_train = np.zeros((len(self.nc1),100))
        for i in range(len(self.nc1)):
            try:
                emb1 = model_word_emb.wv[self.nc1[i]]
            except KeyError:
                emb1 = np.zeros(100)
            try:
                emb2 = model_word_emb.wv[self.nc2[i]]
            except KeyError:
                emb2 = np.zeros(100)
            
            self.X_train[i] = (emb1+emb2)/2
    
    def build_ff_nn(self):
        self.model = Sequential()
        self.model.add(InputLayer(input_shape=(100,)))
        self.model.add(Dense(len(np.unique(self.output_class))))
        self.model.add(Activation('softmax'))
        self.model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy'])
        self.model.summary()
    def train_ff_nn(self):
        tag2int = {tag: i for i, tag in enumerate(list(np.unique(self.output_class)))}
        y_int=[]
        #print(self.output_class)
        for tag in self.output_class:
            y_int.append(tag2int[tag])
        y_train = tf.keras.utils.to_categorical(y_int,num_classes=len(tag2int))
        
        self.history = self.model.fit(self.X_train, (y_train), batch_size=64, epochs=50,validation_split=0.2)
        
    def prediction(self, df_test):
        y_test = df_test['nc']
        y_test_emb=np.zeros((len(y_test),100))
        for i in range(len(y_test)):
            l = y_test[i].split()
            try:
                emb1 = model_word_emb.wv[l[0]]
            except KeyError:
                emb1 = np.zeros(100)
            try:
                emb2 = model_word_emb.wv[[1]]
            except KeyError:
                emb2 = np.zeros(100)
            y_test_emb[i] = (emb1+emb2)/2
        y_pred = self.model.predict(y_test_emb)
        y_pred_tag=[]
        int2tag = {i: tag for i, tag in enumerate(list(np.unique(self.output_class)))}
        for i in y_pred:
            max_index = np.argmax(i)
            tag_pred = int2tag[max_index]
            y_pred_tag.append(tag_pred)
        return y_pred_tag

In [10]:
M = feed_forwrd(df)
M.build_ff_nn()
M.get_embedding()
M.train_ff_nn()
#y_pred = M.predict(preprocessed_test_reviews)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 37)                3737      
_________________________________________________________________
activation (Activation)      (None, 37)                0         
Total params: 3,737
Trainable params: 3,737
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
204/204 [==============================] - 0s 2ms/step - loss: 3.0690 - accuracy: 0.1660 - val_loss: 2.8939 - val_accuracy: 0.2122
Epoch 2/50
204/204 [==============================] - 0s 1ms/step - loss: 2.8483 - accuracy: 0.2169 - val_loss: 2.8033 - val_accuracy: 0.2201
Epoch 3/50
204/204 [==============================] - 0s 1ms/step - loss: 2.7835 - accuracy: 0.2241 - val_loss: 2.7470 - val_accuracy: 0.2247
Epoch 4/50
204/204 [==============================] - 0s 1ms/step - loss: 2.7420 - accurac

In [11]:
y_pred = M.prediction(df3)

In [12]:
df_test = pd.DataFrame(columns=['nc','context','label'])

In [13]:
df_test['nc']=df3['nc']
df_test['context']=df3['context']
df_test['label']=y_pred


In [14]:
df_test.to_json('test_label_part1.json')